## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

July 14 July 20


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,July-18-2025,Condo/Co-op,2528 Boundbrook Dr S #206,Palm Springs,FL,33406.0,119000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2528...,Beaches MLS,RX-11068921,N,Y,26.63772,-80.080643


In [8]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-15    33
2025-07-18    19
2025-07-16    18
2025-07-17     7
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
55,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632
46,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Beaches MLS,RX-11062570,N,Y,26.901317,-80.093391
40,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,MARMLS,A11768038,N,Y,26.380315,-80.159363
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11108088,N,Y,26.389260,-80.067914
95,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10928460,N,Y,26.408119,-80.066261
81,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Beaches MLS,RX-11056144,N,Y,26.586773,-80.037729
77,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3700-...,Beaches MLS,RX-11059619,N,Y,26.408980,-80.065512
94,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2575-...,Beaches MLS,F10413400,N,Y,26.427622,-80.062811
2,PAST SALE,2025-07-16,Condo/Co-op,900 NE Spanish River Blvd Unit 1g,Boca Raton,FL,33431.0,745000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/900-NE-Sp...,Beaches MLS,RX-11108961,N,Y,26.385804,-80.071783
38,PAST SALE,2025-07-15,Condo/Co-op,250 NE 3rd Ave #1308,Delray Beach,FL,33444.0,745000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-NE-...,Beaches MLS,RX-11088198,N,Y,26.466213,-80.070415


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,...,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None
1,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,...,-80.093391,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None
2,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,...,-80.159363,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,David Lane,"Compass Florida, LLC",None,None,None,None,None,None
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,...,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Jamie Moody,William Raveis Real Estate,None,None,Jamie Moody,William Raveis Real Estate,None,None
4,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,...,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Erik Nissani,Lang Realty/BR,None,None,Jessica Grossman,LoKation,None,None
5,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,...,-80.037729,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Dael Sidney,Income Properties South,None,None,Jennifer Noel,RE/MAX Gold,None,None
6,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,...,-80.065512,https://www.redfin.com/FL/Highland-Beach/3700-...,Jason Morss,Tangent Realty Corp,None,None,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR
7,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,...,-80.062811,https://www.redfin.com/FL/Highland-Beach/2575-...,None,None,None,None,None,None,None,None
8,PAST SALE,2025-07-16,Condo/Co-op,900 NE Spanish River Blvd Unit 1g,Boca Raton,FL,33431.0,745000.0,2.0,2.0,...,-80.071783,https://www.redfin.com/FL/Boca-Raton/900-NE-Sp...,Lauren Lispi,RE/MAX Select Group,None,None,Lynne Gewant,Coldwell Banker Realty /Delray Beach,None,None
9,PAST SALE,2025-07-15,Condo/Co-op,250 NE 3rd Ave #1308,Delray Beach,FL,33444.0,745000.0,2.0,2.0,...,-80.070415,https://www.redfin.com/FL/Delray-Beach/250-NE-...,Patricia Burke,One Sotheby's International Realty,Carol Corey,One Sotheby's International Realty,Patrick Meyer,One Sotheby's International Realty,Stephanie Heyman,One Sotheby's International Realty


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

77


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$445,079


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$265


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$34,271,080


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           77              445079          265                     34271080.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
59
-----------
Input Previous Week Condo Average Sales Price:
420076
-----------
Input Previous Week Condo Average PSF:
293
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
24784500.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange
1,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,Admirals Cove,2460.0,NaN,1989.0,NaN,850.0,2670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Beaches MLS,RX-11062570,N,Y,26.901317,-80.093391,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None,2,blue
2,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,FAIRWAY POINT,3348.0,NaN,1988.0,NaN,568.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,MARMLS,A11768038,N,Y,26.380315,-80.159363,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,David Lane,"Compass Florida, LLC",None,None,None,None,None,None,3,blue
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Sea Ranch Club OF Boca I Condo,2421.0,NaN,1981.0,NaN,743.0,2982.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11108088,N,Y,26.389260,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Jamie Moody,William Raveis Real Estate,None,None,Jamie Moody,William Raveis Real Estate,None,None,4,blue
4,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,Toscana West Condo,2292.0,NaN,2001.0,NaN,582.0,2882.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10928460,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Erik Nissani,Lang Realty/BR,None,None,Jessica Grossman,LoKation,None,None,5,blue
5,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,LA Pensee Condo,2115.0,NaN,1985.0,NaN,558.0,2510.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Beaches MLS,RX-11056144,N,Y,26.586773,-80.037729,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Dael Sidney,Income Properties South,None,None,Jennifer Noel,RE/MAX Gold,None,None,6,blue
6,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,Toscana North Condo,2231.0,NaN,2000.0,NaN,502.0,2680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3700-...,Beaches MLS,RX-11059619,N,Y,26.408980,-80.065512,https://www.redfin.com/FL/Highland-Beach/3700-...,Jason Morss,Tangent Realty Corp,None,None,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,7,blue
7,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,Townhouses Of Highland Be,2240.0,NaN,1981.0,NaN,446.0,2485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2575-...,Beaches MLS,F10413400,N,Y,26.427622,-80.062811,https://www.redfin.com/FL/Highland-Beach/2575-...,None,None,None,None,None,None,None,None,8,blue
8,PAST SALE,2025-07-16,Condo/Co-op,900 NE Spanish River Blvd Unit 1g,Boca Raton,FL,33431.0,745000.0,2.0,2.0,Rogers East Condo,1251.0,NaN,1975.0,NaN,596.0,1352.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/900-NE-Sp...,Beaches MLS,RX-11108961,N,Y,2

In [35]:
%store -r map_box_api_key

In [36]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_07212025 


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'Mizner Grand Condo at 550 SE 5th Ave Unit 203s in Boca Raton'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange,Mizner Grand Condo at 550 SE 5th Ave Unit 203s...
1,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,Admirals Cove,2460.0,NaN,1989.0,NaN,850.0,2670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Beaches MLS,RX-11062570,N,Y,26.901317,-80.093391,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None,2,blue,Admirals Cove at 156 Eagle Dr in Jupiter
2,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,FAIRWAY POINT,3348.0,NaN,1988.0,NaN,568.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,MARMLS,A11768038,N,Y,26.380315,-80.159363,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,David Lane,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,FAIRWAY POINT at 20310 Fairway Oaks Dr #173 in...
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Sea Ranch Club OF Boca I Condo,2421.0,NaN,1981.0,NaN,743.0,2982.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11108088,N,Y,26.389260,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Jamie Moody,William Raveis Real Estate,None,None,Jamie Moody,William Raveis Real Estate,None,None,4,blue,Sea Ranch Club OF Boca I Condo at 4301 N Ocean...
4,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,Toscana West Condo,2292.0,NaN,2001.0,NaN,582.0,2882.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10928460,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Erik Nissani,Lang Realty/BR,None,None,Jessica Grossman,LoKation,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd #706 i...
5,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,LA Pensee Condo,2115.0,NaN,1985.0,NaN,558.0,2510.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Beaches MLS,RX-11056144,N,Y,26.586773,-80.037729,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Dael Sidney,Income Properties South,None,None,Jennifer Noel,RE/MAX Gold,None,None,6,blue,LA Pensee Condo at 4000 S Ocean Blvd #401 in S...
6,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,Toscana North Condo,2231.0,NaN,2000.0,NaN,502.0,2680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3700-...,Beaches MLS,RX-11059619,N,Y,26.408980,-80.065512,https://www.redfin.com/FL/Highland-Beach/3700-...,Jason Morss,Tangent Realty Corp,None,None,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,7,blue,Toscana North Condo at 3700 S Ocean Blvd #208 ...
7,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,Townhouses Of Highland Be,2240.0,NaN,1981.0,NaN,446.0,2485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2575-...,Beaches MLS,F10413400,N,Y,26.427622,-80.062811

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name.strip()}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Mizner Grand Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $745,000 to $5,500,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, South Palm Beach, Highland Beach, Boca Raton, Jupiter, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 77 condo sales totaling $34,271,080 million from July 14 to July 20. The previous week, brokers closed 59 condo sales totaling $24,784,500.

Last week’s units sold for an average of $445,079, higher than the $420,076 average price f

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange,Mizner Grand Condo at 550 SE 5th Ave Unit 203s...
1,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,Admirals Cove,2460.0,NaN,1989.0,NaN,850.0,2670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Beaches MLS,RX-11062570,N,Y,26.901317,-80.093391,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None,2,blue,Admirals Cove at 156 Eagle Dr in Jupiter
2,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,FAIRWAY POINT,3348.0,NaN,1988.0,NaN,568.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,MARMLS,A11768038,N,Y,26.380315,-80.159363,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,David Lane,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,FAIRWAY POINT at 20310 Fairway Oaks Dr #173 in...
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Sea Ranch Club OF Boca I Condo,2421.0,NaN,1981.0,NaN,743.0,2982.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11108088,N,Y,26.389260,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Jamie Moody,William Raveis Real Estate,None,None,Jamie Moody,William Raveis Real Estate,None,None,4,blue,Sea Ranch Club OF Boca I Condo at 4301 N Ocean...
4,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,Toscana West Condo,2292.0,NaN,2001.0,NaN,582.0,2882.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10928460,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Erik Nissani,Lang Realty/BR,None,None,Jessica Grossman,LoKation,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd #706 i...
5,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,LA Pensee Condo,2115.0,NaN,1985.0,NaN,558.0,2510.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Beaches MLS,RX-11056144,N,Y,26.586773,-80.037729,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Dael Sidney,Income Properties South,None,None,Jennifer Noel,RE/MAX Gold,None,None,6,blue,LA Pensee Condo at 4000 S Ocean Blvd #401 in S...
6,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,Toscana North Condo,2231.0,NaN,2000.0,NaN,502.0,2680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3700-...,Beaches MLS,RX-11059619,N,Y,26.408980,-80.065512,https://www.redfin.com/FL/Highland-Beach/3700-...,Jason Morss,Tangent Realty Corp,None,None,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,7,blue,Toscana North Condo at 3700 S Ocean Blvd #208 ...
7,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,Townhouses Of Highland Be,2240.0,NaN,1981.0,NaN,446.0,2485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2575-...,Beaches MLS,F10413400,N,Y,26.427622,-80.062811

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Mizner Grand Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $745,000 to $5,500,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, South Palm Beach, Highland Beach, Boca Raton, Jupiter, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 77 condo sales totaling $34,271,080 million from July 14 to July 20. The previous week, brokers closed 59 condo sales totaling $24,784,500.

Last week’s units sold for an average of $445,079, higher than the $420,076 average price f

In [62]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/550-SE-5th-Ave-33432/unit-203S/home/42534955


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Jupiter/156-Eagle-Dr-33477/home/42413326


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/250-NE-3rd-Ave-33444/unit-1308/home/42605380


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/900-NE-Spanish-River-Blvd-33431/unit-1-G/home/178709907


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/550-SE-5th-Ave-33432/unit-203S/home/42534955


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Highland-Beach/2575-S-Ocean-Blvd-33487/unit-305S/home/42388783


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,550 SE 5th Ave Unit 203s,Boca Raton,FL,33432.0,5500000.0,4.0,5.5,Mizner Grand Condo,5835.0,NaN,1999.0,NaN,943.0,6684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,Beaches MLS,RX-11058223,N,Y,26.343864,-80.077632,https://www.redfin.com/FL/Boca-Raton/550-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,David Hall,Compass Florida LLC,None,None,1,orange,Mizner Grand Condo at 550 SE 5th Ave Unit 203s...
1,PAST SALE,2025-07-15,Condo/Co-op,156 Eagle Dr,Jupiter,FL,33477.0,2090000.0,2.0,2.0,Admirals Cove,2460.0,NaN,1989.0,NaN,850.0,2670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Beaches MLS,RX-11062570,N,Y,26.901317,-80.093391,https://www.redfin.com/FL/Jupiter/156-Eagle-Dr...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None,2,blue,Admirals Cove at 156 Eagle Dr in Jupiter
2,PAST SALE,2025-07-15,Condo/Co-op,20310 Fairway Oaks Dr #173,Boca Raton,FL,33434.0,1900000.0,3.0,3.0,FAIRWAY POINT,3348.0,NaN,1988.0,NaN,568.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,MARMLS,A11768038,N,Y,26.380315,-80.159363,https://www.redfin.com/FL/Boca-Raton/20310-Fai...,David Lane,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,FAIRWAY POINT at 20310 Fairway Oaks Dr #173 in...
3,PAST SALE,2025-07-16,Condo/Co-op,4301 N Ocean Blvd #1605,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Sea Ranch Club OF Boca I Condo,2421.0,NaN,1981.0,NaN,743.0,2982.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11108088,N,Y,26.389260,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Jamie Moody,William Raveis Real Estate,None,None,Jamie Moody,William Raveis Real Estate,None,None,4,blue,Sea Ranch Club OF Boca I Condo at 4301 N Ocean...
4,PAST SALE,2025-07-15,Condo/Co-op,3720 S Ocean Blvd #706,Highland Beach,FL,33487.0,1333750.0,2.0,2.5,Toscana West Condo,2292.0,NaN,2001.0,NaN,582.0,2882.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10928460,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Erik Nissani,Lang Realty/BR,None,None,Jessica Grossman,LoKation,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd #706 i...
5,PAST SALE,2025-07-17,Condo/Co-op,4000 S Ocean Blvd #401,South Palm Beach,FL,33480.0,1180000.0,3.0,3.5,LA Pensee Condo,2115.0,NaN,1985.0,NaN,558.0,2510.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Beaches MLS,RX-11056144,N,Y,26.586773,-80.037729,https://www.redfin.com/FL/Palm-Beach/4000-S-Oc...,Dael Sidney,Income Properties South,None,None,Jennifer Noel,RE/MAX Gold,None,None,6,blue,LA Pensee Condo at 4000 S Ocean Blvd #401 in S...
6,PAST SALE,2025-07-17,Condo/Co-op,3700 S Ocean Blvd #208,Highland Beach,FL,33487.0,1120000.0,2.0,2.5,Toscana North Condo,2231.0,NaN,2000.0,NaN,502.0,2680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3700-...,Beaches MLS,RX-11059619,N,Y,26.408980,-80.065512,https://www.redfin.com/FL/Highland-Beach/3700-...,Jason Morss,Tangent Realty Corp,None,None,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,7,blue,Toscana North Condo at 3700 S Ocean Blvd #208 ...
7,PAST SALE,2025-07-16,Condo/Co-op,2575 S Ocean Blvd Unit 305S,Highland Beach,FL,33487.0,1000000.0,2.0,2.5,Townhouses Of Highland Be,2240.0,NaN,1981.0,NaN,446.0,2485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2575-...,Beaches MLS,F10413400,N,Y,26.427622,-80.062811

## Time on Market Calculator

In [65]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 2, 14) ## List (Earlier) date
date2 = datetime(2025, 7, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

151


## Clean CSV for Datawrapper Chart

In [66]:
chart_df = df_top_ten

In [67]:
chart_df = chart_df.fillna(" ")

In [68]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [69]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [70]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [71]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [72]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [73]:
chart_df['PPSF'].astype(float)

0    943.0
1    850.0
2    568.0
3    743.0
4    582.0
5    558.0
6    502.0
7    446.0
8    596.0
9    604.0
Name: PPSF, dtype: float64

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [75]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [78]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-15,Mizner Grand Condo,550 SE 5th Ave Unit 203s in Boca Raton,"$5,500,000","5,835",$943,1999,David Hall Compass Florida LLC Sonia Hall Comp...,David Hall Compass Florida LLC
1,2025-07-15,Admirals Cove,156 Eagle Dr in Jupiter,"$2,090,000","2,460",$850,1989,Thomas Frankel Admirals Cove Realty Co Inc All...,Thomas Frankel Admirals Cove Realty Co Inc
2,2025-07-15,FAIRWAY POINT,20310 Fairway Oaks Dr #173 in Boca Raton,"$1,900,000","3,348",$568,1988,"David Lane Compass Florida, LLC",
3,2025-07-16,Sea Ranch Club OF Boca I Condo,4301 N Ocean Blvd #1605 in Boca Raton,"$1,800,000","2,421",$743,1981,Jamie Moody William Raveis Real Estate,Jamie Moody William Raveis Real Estate
4,2025-07-15,Toscana West Condo,3720 S Ocean Blvd #706 in Highland Beach,"$1,333,750","2,292",$582,2001,Erik Nissani Lang Realty/BR,Jessica Grossman LoKation
5,2025-07-17,LA Pensee Condo,4000 S Ocean Blvd #401 in South Palm Beach,"$1,180,000","2,115",$558,1985,Dael Sidney Income Properties South,Jennifer Noel RE/MAX Gold
6,2025-07-17,Toscana North Condo,3700 S Ocean Blvd #208 in Highland Beach,"$1,120,000","2,231",$502,2000,Jason Morss Tangent Realty Corp,Judith Randon Judith Randon Realty Nancy Ghen ...
7,2025-07-16,Townhouses Of Highland Be,2575 S Ocean Blvd Unit 305S in Highland Beach,"$1,000,000","2,240",$446,1981,,
8,2025-07-16,Rogers East Condo,900 NE Spanish River Blvd Unit 1g in Boca Raton,"$745,000","1,251",$596,1975,Lauren Lispi RE/MAX Select Group,Lynne Gewant Coldwell Banker Realty /Delray Be...
9,2025-07-15,Pineapple Grove Village Condo,250 NE 3rd Ave #1308 in Delray Beach,"$745,000","1,234",$604,2004,Patricia Burke One Sotheby's International Rea...,Patrick Meyer One Sotheby's International Real...
